## Analyze Crash Data
* Analyzinginjuries and fatalities of drivers, cyclists and pedestrians in Washington, DC.
* Statistics and insights into the types of crashes.
* Geolocation analysis to determine the streets, wards and quadrants that are most prone to accidents.

Data provided by Open Data DC: http://opendata.dc.gov/datasets/crashes-in-dc 

In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm_notebook as tqdm

In [3]:
crashes = pd.read_csv('data/Crashes_in_DC.csv')

C:\Users\Himanshu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
bad_cols= ['LOCATIONERROR', 'LASTUPDATEDATE', 'TODATE', 'MPDGEOX', 'MPDGEOY']
for col in bad_cols:
    crashes.drop(col, inplace=True, axis=1)

## Analyze INJURIES, FATALITIES and IMPAIRMENTS by LOCATION  
### Location Factors
* Coordinates
* Street Name
* Ward

In [6]:
crashes.shape

(204291, 49)

In [7]:
location_cols = ['WARD', 'ROUTEID', 'NEARESTINTSTREETNAME', 'INTAPPROACHDIRECTION']

event_cols = ['MAJORINJURIES_BICYCLIST',
 'MINORINJURIES_BICYCLIST',
 'UNKNOWNINJURIES_BICYCLIST',
 'FATAL_BICYCLIST',
 'MAJORINJURIES_DRIVER',
 'MINORINJURIES_DRIVER',
 'UNKNOWNINJURIES_DRIVER',
 'FATAL_DRIVER',
 'MAJORINJURIES_PEDESTRIAN',
 'MINORINJURIES_PEDESTRIAN',
 'UNKNOWNINJURIES_PEDESTRIAN',
 'FATAL_PEDESTRIAN',]

issues = {}
for col in location_cols:
    issues.setdefault(col, {})
    print(f"Number of unique {col}: {len(crashes[col].unique())}")
    for location in tqdm(crashes[col].unique(), total=len(crashes[col].unique())):
        loc_df = crashes[crashes[col]==location]
        issues[col].setdefault(location, {})
        issues[col][location]['TOTAL_EVENTS'] = loc_df.shape[0]
        for event in event_cols:
            issues[col][location].setdefault(event, loc_df[event].sum())

Number of unique WARD: 10



Number of unique ROUTEID: 8194



Number of unique NEARESTINTSTREETNAME: 2124



Number of unique INTAPPROACHDIRECTION: 9


In [44]:
def create_stats_df(issues_dict, location, rate):
    df = pd.DataFrame(issues_dict[location]).T
    if rate==True:
        for col in df:
            df[f'{col}_RATE'] = (df[col] / df['TOTAL_EVENTS'])*100
            
    # fatalities
    df['TOTAL_FATAL'] = df['FATAL_BICYCLIST'] + df['FATAL_DRIVER'] + df['FATAL_PEDESTRIAN']
    # major injuries
    df['TOTAL_MAJORINJURIES'] = df['MAJORINJURIES_BICYCLIST'] + df['MAJORINJURIES_DRIVER'] + df['MAJORINJURIES_PEDESTRIAN']
    # minor injuries
    df['TOTAL_MINORINJURIES'] = df['MINORINJURIES_BICYCLIST'] + df['MINORINJURIES_DRIVER'] + df['MINORINJURIES_PEDESTRIAN']
    # unknown injuries
    df['TOTAL_UNKNOWNINJURIES'] = df['UNKNOWNINJURIES_BICYCLIST'] + df['UNKNOWNINJURIES_DRIVER'] + df['UNKNOWNINJURIES_PEDESTRIAN']
    
    # total accidents
    df['TOTAL_ACCIDENTS'] = df['TOTAL_FATAL'] + df['TOTAL_MAJORINJURIES'] + df['TOTAL_MINORINJURIES'] + df['TOTAL_UNKNOWNINJURIES']
    
    return df

## Create interactive maps of accidents

### 1. Ward level

In [12]:
import folium

In [10]:
wards_df = create_stats_df(issues, 'WARD', rate=True)
wards_df.fillna("", inplace=True)
wards_df

,FATAL_BICYCLIST,FATAL_DRIVER,FATAL_PEDESTRIAN,MAJORINJURIES_BICYCLIST,MAJORINJURIES_DRIVER,MAJORINJURIES_PEDESTRIAN,MINORINJURIES_BICYCLIST,MINORINJURIES_DRIVER,MINORINJURIES_PEDESTRIAN,TOTAL_EVENTS,...,MAJORINJURIES_BICYCLIST_RATE,MAJORINJURIES_DRIVER_RATE,MAJORINJURIES_PEDESTRIAN_RATE,MINORINJURIES_BICYCLIST_RATE,MINORINJURIES_DRIVER_RATE,MINORINJURIES_PEDESTRIAN_RATE,TOTAL_EVENTS_RATE,UNKNOWNINJURIES_BICYCLIST_RATE,UNKNOWNINJURIES_DRIVER_RATE,UNKNOWNINJURIES_PEDESTRIAN_RATE
Ward 6,1,23,1,35,2169,68,266,4987,454,29962,...,0.116815,7.23917,0.226954,0.887791,16.6444,1.51525,100,0.00667512,0.814365,0.0433883
Ward 2,2,27,6,100,3191,147,624,5623,825,46730,...,0.213995,6.82859,0.314573,1.33533,12.033,1.76546,100,0.0149797,0.727584,0.0256794
Ward 7,2,35,4,8,2426,83,55,5204,296,24961,...,0.03205,9.71916,0.332519,0.220344,20.8485,1.18585,100,0.0080125,1.32607,0.016025
Ward 3,0,4,0,8,1080,39,75,2333,187,13190,...,0.060652,8.18802,0.295679,0.568613,17.6876,1.41774,100,0,0.515542,0.0075815
Ward 5,1,30,3,32,2831,62,205,6200,385,30682,...,0.104296,9.22691,0.202073,0.668144,20.2073,1.25481,100,0,0.837625,0.0260739
Ward 1,3,13,2,49,1669,77,322,2401,346,18315,...,0.26754,9.11275,0.42042,1.75812,13.1095,1.88916,100,0.02184,0.698881,0.01638
Ward 4,0,12,3,24,1726,56,143,3643,260,17815,...,0.134718,9.68846,0.314342,0.802694,20.4491,1.45944,100,0,0.493966,0.00561325
Ward 8,1,20,1,8,1797,77,55,4144,347,21248,...,0.0376506,8.45727,0.362387,0.258848,19.503,1.63309,100,0.00470633,1.77428,0.0235316
NaN,0,0,0,0,0,0,0,0,0,0,...,,,,,,,,,,
Null,0,0,0,0,192,16,1,271,25,1386,...,0,13.8528,1.1544,0.0721501,19.5527,1.80375,100,0,0,0


In [11]:
wards_dict = wards_df.to_dict(orient='index')
with open('data/ward_crash_data.json', 'w') as outfile:
    json.dump(wards_dict, outfile)

In [13]:
ward_geo = 'data/dc_wards.json'

#### Lets test by plotting `MINORINJURIES_DRIVER_RATE`

In [14]:
wards_df.reset_index(inplace=True)
wards_df.rename(columns={'index': 'Ward'}, inplace=True)
wards_df = wards_df[wards_df['Ward'].isnull()==False]
wards_df = wards_df[wards_df['Ward']!='Null']
wards_df

,Ward,FATAL_BICYCLIST,FATAL_DRIVER,FATAL_PEDESTRIAN,MAJORINJURIES_BICYCLIST,MAJORINJURIES_DRIVER,MAJORINJURIES_PEDESTRIAN,MINORINJURIES_BICYCLIST,MINORINJURIES_DRIVER,MINORINJURIES_PEDESTRIAN,...,MAJORINJURIES_BICYCLIST_RATE,MAJORINJURIES_DRIVER_RATE,MAJORINJURIES_PEDESTRIAN_RATE,MINORINJURIES_BICYCLIST_RATE,MINORINJURIES_DRIVER_RATE,MINORINJURIES_PEDESTRIAN_RATE,TOTAL_EVENTS_RATE,UNKNOWNINJURIES_BICYCLIST_RATE,UNKNOWNINJURIES_DRIVER_RATE,UNKNOWNINJURIES_PEDESTRIAN_RATE
0,Ward 6,1,23,1,35,2169,68,266,4987,454,...,0.116815,7.23917,0.226954,0.887791,16.6444,1.51525,100,0.00667512,0.814365,0.0433883
1,Ward 2,2,27,6,100,3191,147,624,5623,825,...,0.213995,6.82859,0.314573,1.33533,12.033,1.76546,100,0.0149797,0.727584,0.0256794
2,Ward 7,2,35,4,8,2426,83,55,5204,296,...,0.03205,9.71916,0.332519,0.220344,20.8485,1.18585,100,0.0080125,1.32607,0.016025
3,Ward 3,0,4,0,8,1080,39,75,2333,187,...,0.060652,8.18802,0.295679,0.568613,17.6876,1.41774,100,0,0.515542,0.0075815
4,Ward 5,1,30,3,32,2831,62,205,6200,385,...,0.104296,9.22691,0.202073,0.668144,20.2073,1.25481,100,0,0.837625,0.0260739
5,Ward 1,3,13,2,49,1669,77,322,2401,346,...,0.26754,9.11275,0.42042,1.75812,13.1095,1.88916,100,0.02184,0.698881,0.01638
6,Ward 4,0,12,3,24,1726,56,143,3643,260,...,0.134718,9.68846,0.314342,0.802694,20.4491,1.45944,100,0,0.493966,0.00561325
7,Ward 8,1,20,1,8,1797,77,55,4144,347,...,0.0376506,8.45727,0.362387,0.258848,19.503,1.63309,100,0.00470633,1.77428,0.0235316


In [17]:
chosen_metric = 'MINORINJURIES_DRIVER_RATE'

In [18]:
data_to_plot = wards_df[['Ward', chosen_metric]]
data_to_plot[chosen_metric] = data_to_plot[chosen_metric].astype(float)

dc_crash_map = folium.Map(location=[38.9072, -77.0369], zoom_start=11)
dc_crash_map.choropleth(geo_data=ward_geo, 
                        data=data_to_plot,
                        columns=['Ward', chosen_metric],
                        key_on='feature.properties.name',
                        fill_color='OrRd', 
                        fill_opacity=0.7, 
                        line_opacity=0.2,
                        legend_name=f"Rate of {chosen_metric} in DC"
                       )

# Create Folium plot
dc_crash_map.save(f'{chosen_metric}.html')
dc_crash_map.save(f'{chosen_metric}.svg')
# Import the Folium interactive html file
from IPython.display import HTML
HTML(f'<iframe src={chosen_metric}.html width=500 height=450></iframe>')

C:\Users\Himanshu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Himanshu\Anaconda3\lib\site-packages\folium\folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


### 2. Quadrant Level (TODO)

In [9]:
direction_df = create_stats_df(issues, 'INTAPPROACHDIRECTION', rate=True)
direction_df

,FATAL_BICYCLIST,FATAL_DRIVER,FATAL_PEDESTRIAN,MAJORINJURIES_BICYCLIST,MAJORINJURIES_DRIVER,MAJORINJURIES_PEDESTRIAN,MINORINJURIES_BICYCLIST,MINORINJURIES_DRIVER,MINORINJURIES_PEDESTRIAN,TOTAL_EVENTS,...,MAJORINJURIES_BICYCLIST_RATE,MAJORINJURIES_DRIVER_RATE,MAJORINJURIES_PEDESTRIAN_RATE,MINORINJURIES_BICYCLIST_RATE,MINORINJURIES_DRIVER_RATE,MINORINJURIES_PEDESTRIAN_RATE,TOTAL_EVENTS_RATE,UNKNOWNINJURIES_BICYCLIST_RATE,UNKNOWNINJURIES_DRIVER_RATE,UNKNOWNINJURIES_PEDESTRIAN_RATE
NaN,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
North,3,48,7,74,5945,287,400,10336,884,57237,...,0.129287,10.386638,0.501424,0.698849,18.058249,1.544456,100.0,0.00000,0.232367,0.003494
South,0,25,3,56,2438,82,369,4896,576,31141,...,0.179827,7.828907,0.263318,1.184933,15.722038,1.849652,100.0,0.00000,0.234418,0.000000
East,0,18,2,31,1884,48,187,3314,283,21784,...,0.142306,8.648549,0.220345,0.858428,15.213000,1.299119,100.0,0.00000,0.339699,0.009181
Northwest,0,14,1,7,844,13,64,1684,114,9380,...,0.074627,8.997868,0.138593,0.682303,17.953092,1.215352,100.0,0.00000,0.330490,0.010661
Southeast,0,5,1,10,720,22,61,1604,121,8876,...,0.112663,8.111762,0.247859,0.687247,18.071203,1.363227,100.0,0.00000,0.349256,0.011266
Southwest,0,15,1,8,771,17,72,1987,132,10161,...,0.078732,7.587836,0.167306,0.708592,19.555162,1.299085,100.0,0.00000,0.364137,0.009842
Northeast,1,11,0,8,903,28,56,2090,114,10423,...,0.076753,8.663533,0.268637,0.537273,20.051809,1.093735,100.0,0.00000,0.393361,0.000000
West,3,11,2,41,2600,50,336,5282,520,34637,...,0.118371,7.506424,0.144354,0.970061,15.249589,1.501285,100.0,0.02021,1.804429,0.046193


## Most dangerous streets

In [45]:
street_df = create_stats_df(issues, 'NEARESTINTSTREETNAME', rate=True)
street_df

,FATAL_BICYCLIST,FATAL_DRIVER,FATAL_PEDESTRIAN,MAJORINJURIES_BICYCLIST,MAJORINJURIES_DRIVER,MAJORINJURIES_PEDESTRIAN,MINORINJURIES_BICYCLIST,MINORINJURIES_DRIVER,MINORINJURIES_PEDESTRIAN,TOTAL_EVENTS,...,MINORINJURIES_PEDESTRIAN_RATE,TOTAL_EVENTS_RATE,UNKNOWNINJURIES_BICYCLIST_RATE,UNKNOWNINJURIES_DRIVER_RATE,UNKNOWNINJURIES_PEDESTRIAN_RATE,TOTAL_FATAL,TOTAL_MAJORINJURIES,TOTAL_MINORINJURIES,TOTAL_UNKNOWNINJURIES,TOTAL_ACCIDENTS
NaN,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
PENNSYLVANIA AVE NW,0,0,0,2,44,7,13,112,23,931,...,2.470462,100.0,0.000000,0.429646,0.000000,0,53,148,4,205
MASSACHUSETTS AVE NW,1,0,0,1,104,2,15,246,23,1571,...,1.464036,100.0,0.063654,0.700191,0.063654,1,107,284,13,405
44TH ST NW,0,0,0,0,13,0,2,19,3,171,...,1.754386,100.0,0.000000,0.000000,0.000000,0,13,24,0,37
7TH ST NE,0,0,0,0,50,3,4,77,4,480,...,0.833333,100.0,0.000000,1.041667,0.000000,0,53,85,5,143
COLUMBIA RD NW,0,0,0,3,80,3,19,101,23,903,...,2.547065,100.0,0.000000,0.332226,0.000000,0,86,143,3,232
M ST SW,0,0,0,0,19,0,6,47,7,313,...,2.236422,100.0,0.000000,0.958466,0.000000,0,19,60,3,82
DUPONT CIR NW,0,0,0,0,26,0,5,23,4,392,...,1.020408,100.0,0.000000,0.510204,0.000000,0,26,32,2,60
CANAL RD NW,0,0,0,0,11,0,0,28,2,129,...,1.550388,100.0,0.000000,2.325581,0.000000,0,11,30,3,44
4TH ST SW,0,0,0,0,19,1,5,91,4,405,...,0.987654,100.0,0.000000,1.728395,0.000000,0,20,100,7,127


In [50]:
top10_fatal_streets = street_df['TOTAL_FATAL'].sort_values(ascending=False).head(10)
top10_major_streets = street_df['TOTAL_MAJORINJURIES'].sort_values(ascending=False).head(10)
top10_minor_streets = street_df['TOTAL_MINORINJURIES'].sort_values(ascending=False).head(10)
top10_total_streets = street_df['TOTAL_ACCIDENTS'].sort_values(ascending=False).head(10)

In [51]:
top10_fatal_streets

H ST NE                4
EUCLID ST NW           3
18TH ST NE             3
FLORIDA AVE NE         3
ALABAMA AVE SE         3
PENNSYLVANIA AVE SE    3
INDEPENDENCE AVE SW    3
GOOD HOPE RD SE        3
HOWARD RD SE           3
MINNESOTA AVE SE       2
Name: TOTAL_FATAL, dtype: int64

In [52]:
top10_major_streets

14TH ST NW             308
M ST NW                195
GEORGIA AVE NW         189
WISCONSIN AVE NW       186
13TH ST NW             178
PENNSYLVANIA AVE SE    177
16TH ST NW             166
H ST NW                157
SOUTHERN AVE SE        157
7TH ST NW              157
Name: TOTAL_MAJORINJURIES, dtype: int64

In [53]:
top10_minor_streets

14TH ST NW             494
GEORGIA AVE NW         464
SOUTHERN AVE SE        416
PENNSYLVANIA AVE SE    397
M ST NW                365
K ST NW                357
H ST NW                356
NEW YORK AVE NW        331
16TH ST NW             324
ALABAMA AVE SE         307
Name: TOTAL_MINORINJURIES, dtype: int64

In [54]:
top10_total_streets

14TH ST NW             822
GEORGIA AVE NW         662
SOUTHERN AVE SE        600
PENNSYLVANIA AVE SE    596
M ST NW                583
H ST NW                525
K ST NW                503
16TH ST NW             503
WISCONSIN AVE NW       493
13TH ST NW             476
Name: TOTAL_ACCIDENTS, dtype: int64